### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several reasons to consider using AutoGen in your new AI Agent project:

1. **Enhanced Debugging Capabilities**: AutoGen provides transparent and replayable traces of decisions made during interactions, which can significantly simplify debugging processes compared to traditional opaque stack traces.

2. **Ease of Use**: As an open-source framework, AutoGen is designed to streamline the development of AI agents, making it more accessible for developers to structure their applications.

3. **Multi-Agent Collaboration**: AutoGen allows for the creation of multi-agent systems where different AI models can work together seamlessly, enhancing the capability to tackle complex tasks through agent collaboration.

4. **Integration of Tools**: The framework supports integration with various tools and models, allowing for more flexibility in deploying diverse functionalities within your AI agents.

5. **Scalable Solution**: AutoGen is designed to improve efficiency and scalability, making it suitable for projects that may grow in complexity over time.

6. **Advanced Features**: It combines ideas from related projects, incorporating new capabilities such as improved orchestration of tools and memory management, which can lead to better performance in task execution.

7. **Support for Multiple LLMs**: AutoGen facilitates the use of multiple large language models (LLMs), which can enhance the diversity and effectiveness of the AI solution in different contexts.

Overall, AutoGen offers a robust framework that could significantly enhance the development and functionality of AI Agent projects. 

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI Agent project:

1. **Prompt Engineering Intensity**: AutoGen requires detailed and algorithmic prompts, which can be time-consuming to develop and may increase operational costs as the complexity of prompts scales.

2. **Complexity Overhead**: Utilizing a framework like AutoGen can lead to over-engineered solutions, where the complexity of the system may outweigh the benefits, making maintenance and scaling more challenging.

3. **Steep Learning Curve**: There may be a significant learning curve associated with understanding and effectively using AutoGen, which can delay project timelines.

4. **Resource Intensive**: Because of its focus on multi-agent collaboration, AutoGen can be resource-intensive, requiring more computational power and data than simpler frameworks.

5. **Limited Flexibility**: While AutoGen is powerful, it may also impose constraints on the flexibility of use compared to low-code platforms that allow for more ad-hoc workflows.

6. **Potential for AI Error Propagation**: In multi-agent systems, if one agent makes a mistake, that error can propagate through the system, leading to compounded issues.

These factors should be taken into account when deciding whether to integrate AutoGen into your project. 

TERMINATE



## Decision:

Based on the analysis of the pros and cons, I would recommend using AutoGen for the project. 

**Rationale**: The advantages of AutoGen, such as enhanced debugging capabilities, ease of use as an open-source framework, multi-agent collaboration, and scalability, provide significant benefits for the development of complex AI systems. The ability to integrate with various tools and use multiple LLMs further enriches the potential outcomes of the project. While the cons, including the potential for increased complexity and resource intensity, are valid considerations, the overall benefits align well with the goals of creating a robust and efficient AI agent. The decision to proceed with AutoGen can lead to improved performance and functionality in the long run. 

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()